## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,40.05,75,83,2.24,broken clouds
1,1,Biltine,TD,14.5333,20.9167,79.02,68,100,2.64,overcast clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,57.92,89,89,7.27,overcast clouds
3,3,Esperance,AU,-33.8667,121.9000,53.44,76,90,20.65,light rain
4,4,Qorveh,IR,35.1664,47.8045,67.12,43,0,1.25,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Biltine,TD,14.5333,20.9167,79.02,68,100,2.64,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,84.18,83,90,1.99,light rain
8,8,Rikitea,PF,-23.1203,-134.9692,70.25,58,9,12.35,clear sky
9,9,Sola,VU,-13.8833,167.5500,80.94,75,69,17.54,broken clouds
11,11,Port Hedland,AU,-20.3167,118.5667,77.05,25,4,17.27,clear sky
13,13,Erenhot,CN,43.6475,111.9767,80.08,45,100,2.62,overcast clouds
16,16,Vaini,TO,-21.2000,-175.2000,82.56,83,20,5.75,few clouds
23,23,Itarema,BR,-2.9248,-39.9167,78.87,77,37,15.10,scattered clouds
24,24,Butaritari,KI,3.0707,172.7902,83.26,76,4,16.46,light rain
25,25,Amapa,BR,1.0000,-52.0000,70.50,94,17,2.68,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Biltine,TD,14.5333,20.9167,79.02,68,100,2.64,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,84.18,83,90,1.99,light rain
8,8,Rikitea,PF,-23.1203,-134.9692,70.25,58,9,12.35,clear sky
9,9,Sola,VU,-13.8833,167.5500,80.94,75,69,17.54,broken clouds
11,11,Port Hedland,AU,-20.3167,118.5667,77.05,25,4,17.27,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Biltine,TD,79.02,overcast clouds,14.5333,20.9167,
6,Kapaa,US,84.18,light rain,22.0752,-159.3190,
8,Rikitea,PF,70.25,clear sky,-23.1203,-134.9692,
9,Sola,VU,80.94,broken clouds,-13.8833,167.5500,
11,Port Hedland,AU,77.05,clear sky,-20.3167,118.5667,
13,Erenhot,CN,80.08,overcast clouds,43.6475,111.9767,
16,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
23,Itarema,BR,78.87,scattered clouds,-2.9248,-39.9167,
24,Butaritari,KI,83.26,light rain,3.0707,172.7902,
25,Amapa,BR,70.50,few clouds,1.0000,-52.0000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Biltine,TD,79.02,overcast clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine
6,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Rikitea,PF,70.25,clear sky,-23.1203,-134.9692,People ThankYou
9,Sola,VU,80.94,broken clouds,-13.8833,167.5500,Leumerus Bungalows
11,Port Hedland,AU,77.05,clear sky,-20.3167,118.5667,The Esplanade Hotel
13,Erenhot,CN,80.08,overcast clouds,43.6475,111.9767,Erenhot Rurouni Youth Hostel
16,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Itarema,BR,78.87,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
24,Butaritari,KI,83.26,light rain,3.0707,172.7902,Isles Sunset Lodge
25,Amapa,BR,70.50,few clouds,1.0000,-52.0000,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Biltine,TD,79.02,overcast clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine
6,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Rikitea,PF,70.25,clear sky,-23.1203,-134.9692,People ThankYou
9,Sola,VU,80.94,broken clouds,-13.8833,167.5500,Leumerus Bungalows
11,Port Hedland,AU,77.05,clear sky,-20.3167,118.5667,The Esplanade Hotel
13,Erenhot,CN,80.08,overcast clouds,43.6475,111.9767,Erenhot Rurouni Youth Hostel
16,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Itarema,BR,78.87,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
24,Butaritari,KI,83.26,light rain,3.0707,172.7902,Isles Sunset Lodge
27,Tabou,CI,74.97,overcast clouds,4.4230,-7.3528,hotêl doufoulougou


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather with Max Temp</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))